# Pop Change Factors 15County 2010-2023

## Setup

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import time
from arcgis.features import FeatureLayerCollection
from arcgis.gis import GIS
from pathlib import Path
from zipfile import ZipFile
import pandas as pd

# Connect to ArcGIS Online
gis = GIS("https://www.arcgis.com", "ogwynn_morpc", "Cr33p3r11!!")

C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\dask\dataframe\__init__.py:49: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


## Data inititalization

In [2]:
data = pd.read_csv('popFactors.csv')

# Convert the dataframe from wide to long format
long_data = pd.melt(data, id_vars=['YEAR', 'NAME'], var_name='VARIABLE', value_name='VALUE')

# Save the long-form dataframe to a new CSV file
output_file_path = 'popFactors_long.csv'
long_data.to_csv(output_file_path, index=False)

# Display the first few rows of the long-form dataframe
long_data.head(), output_file_path

(   YEAR              NAME VARIABLE    VALUE
 0  2010  15-County Region   BIRTHS  29733.0
 1  2011  15-County Region   BIRTHS  29705.0
 2  2012  15-County Region   BIRTHS  30059.0
 3  2013  15-County Region   BIRTHS  30594.0
 4  2014  15-County Region   BIRTHS  30746.0,
 'popFactors_long.csv')

## Pulling Existing Catalog from AGO

In [3]:
# Define the directory to save the data
data_path = Path('inputData/insightsCatalog')
data_path.mkdir(parents=True, exist_ok=True)

# Access the CSV item by ID
catalog_item_id = "c396b9d0db724d1f870fa47f1d1dc1ce"
csv_item = gis.content.get(catalog_item_id)

# Download the CSV file
catalog_path = csv_item.download(save_path=data_path)

print(f"CSV file downloaded successfully: {catalog_path}")

CSV file downloaded successfully: inputData\insightsCatalog\morpc_insights_catalog.xlsx


## Creating plots for 15 counties and region summary

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import os

df = long_data

# Create directory to store PNG files
output_dir = "outputData/population_plots"
os.makedirs(output_dir, exist_ok=True)

# Custom legend labels
legend_labels = {
    'BIRTHS': 'Births',
    'DEATHS': 'Deaths',
    'NETMIG': 'Net Migration',
    'NATINC': 'Natural Pop Change'
}

# Get unique county names after filtering
unique_names = df['NAME'].unique()

# Filter data for relevant variables
filtered_df = df[df['VARIABLE'].isin(['BIRTHS', 'DEATHS', 'NATINC', 'NETMIG'])]

# Define color palette
colors = ['#365072', '#0075BF', '#00B2BF', '#2A7F67', '#8DBF00']

# Generate line plots with 16:9 aspect ratio and updated font size
for name in unique_names:
    subset = filtered_df[filtered_df['NAME'] == name]

    # Pivot the data for easier plotting
    pivot_data = subset.pivot(index='YEAR', columns='VARIABLE', values='VALUE')

    plt.figure(figsize=(16, 9))  # Set aspect ratio to 16:9
    for i, col in enumerate(pivot_data.columns):
        plt.plot(pivot_data.index, pivot_data[col], marker='o', label=legend_labels.get(col, col), color=colors[i % len(colors)])

    plt.title(f"What is driving population change in {name} County? (2010-2023)", fontsize=18)

    # Place legend below the plot
    plt.legend(fontsize=15, loc='lower center', bbox_to_anchor=(0.5, -0.2), ncol=2, frameon=True)

    plt.grid(True)
    
    # Remove x and y axis labels
    plt.xlabel("")
    plt.ylabel("")
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)

    # Save each plot in the created directory
    filename = os.path.join(output_dir, f"{name.replace(' ', '_')}_population_factors.png")
    plt.savefig(filename, bbox_inches='tight')  # Ensure legend is fully captured
    plt.close()

# Notify user
f"Updated plots saved in the '{output_dir}' directory with 16:9 aspect ratio and font size 15."


"Updated plots saved in the 'outputData/population_plots' directory with 16:9 aspect ratio and font size 15."

# Upload to AGO

In [5]:
import time
from arcgis.gis import GIS
import os
import pandas as pd

# Directory where images are saved
output_dir = 'outputData/population_plots'

# Initialize list to store county-item mappings
county_item_mapping = []

# Connect to ArcGIS Online
gis = GIS("https://www.arcgis.com", "ogwynn_morpc", "Cr33p3r11!!")

# Loop through the output directory and upload files
for filename in os.listdir(output_dir):
    if filename.endswith(".png"):
        file_path = os.path.join(output_dir, filename)
        
        # Define item properties
        title = filename.replace("_", " ").replace(".png", "")
        county_name = filename.split('_')[0]  # Extract county name from the filename
        item_properties = {
            "title": title,
            "description": f"Population trends in {county_name} County (2010-2023).",
            "type": "Image"
        }
        
        # Search for an existing item with the same title
        existing_items = gis.content.search(query=title, item_type="Image", max_items=1)
        if existing_items:
            # If an existing item is found, delete it
            existing_item = existing_items[0]
            existing_item.delete()
            print(f"Deleted existing item with title: {title}")
            time.sleep(2)  # Wait for 2 seconds before uploading the new item

        try:
            # Upload the new image
            image_item = gis.content.add(item_properties, data=file_path)
            
            # Make the item public
            image_item.share(everyone=True)

            # Append the county name, item URL, and title to the list
            county_item_mapping.append((county_name, f"https://www.arcgis.com/sharing/rest/content/items/{image_item.id}/data", title))

            # Print confirmation
            print(f"Uploaded {filename} with item ID: {image_item.id} and set it to public")
        
        except Exception as e:
            print(f"Error uploading {filename}: {e}")

# Convert the list to a DataFrame and save
mapping_df = pd.DataFrame(county_item_mapping, columns=['COUNTY', 'itemURL', 'title'])
mapping_df['TilesetID'] = 13
mapping_file_path = 'AGO_County_ItemURL_Mapping.csv'
mapping_df.to_csv(mapping_file_path, index=False)

print(f"Upload complete. Mapping file saved as '{mapping_file_path}'.")


Deleted existing item with title: 15-County Region population factors


C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded 15-County_Region_population_factors.png with item ID: ec15ef1c82464f1687fbae796fdb4000 and set it to public
Deleted existing item with title: Delaware population factors


C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded Delaware_population_factors.png with item ID: dedd3778f5534726ab15c35301319b37 and set it to public
Deleted existing item with title: Fairfield population factors


...

Uploaded Pickaway_population_factors.png with item ID: 0e421e856154421481ec269619dfaf5e and set it to public
Deleted existing item with title: Ross population factors


C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded Ross_population_factors.png with item ID: 1287f1deda1d413b93409ff2c8c42a1e and set it to public
Deleted existing item with title: Union population factors


C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded Union_population_factors.png with item ID: 983990630b694935a4e1a7c225f280b1 and set it to public
Upload complete. Mapping file saved as 'AGO_County_ItemURL_Mapping.csv'.


## Updating local catalog

In [7]:
import pandas as pd

# Load the catalog data
catalog_df = pd.read_excel(catalog_path)

# Load the mapping data
mapping_df = pd.read_csv(mapping_file_path)

# Standardize 'COUNTY' values in mapping_df
mapping_df['COUNTY'] = mapping_df['COUNTY'].replace('15-County', '15County')

# Create a mapping dictionary from mapping_df
mapping_dict = dict(zip(zip(mapping_df['COUNTY'], mapping_df['TilesetID']), mapping_df['itemURL']))

# Update 'ThumbnailURL' only where there is a match in mapping_dict
catalog_df['ThumbnailURL'] = catalog_df.apply(
    lambda row: mapping_dict.get((row['GeographyName'], row['TilesetID']), row['ThumbnailURL']),
    axis=1
)

# Save the updated catalog back to an Excel file
updated_catalog_path = "outputData/morpc_insights_catalog_updated.xlsx"
catalog_df.to_excel(updated_catalog_path, index=False)

# Provide the file to the user
print(f"Updated catalog saved at: {updated_catalog_path}")


Updated catalog saved at: outputData/morpc_insights_catalog_updated.xlsx


## Pushing changes to AGO Excel

In [8]:
# Define the item ID of the Excel file in AGO
ITEM_ID = "c396b9d0db724d1f870fa47f1d1dc1ce"

# Get the existing item
item = gis.content.get(ITEM_ID)

if item:
    print(f"Found item: {item.title}")

    # Update the item with the new file
    item.update(
        item_properties={"title": item.title},  # Keeps the existing metadata
        data=updated_catalog_path  # Replaces the file
    )
    
    print("Successfully updated the Excel file in AGO.")
else:
    print("Item not found. Check the item ID and your permissions.")


Found item: morpc_insights_catalog
Successfully updated the Excel file in AGO.
